In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(13)

In [2]:
response_vectors = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
], dtype=np.float32)

data = []

n = 10000
for i in range(n):
    instance = np.array([random.randint(0,1), random.randint(0,1), random.randint(0,1)])
    label = [instance[0]*4 + instance[1]*2 + instance[2]]
    noise = np.array([random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1)])

    data.append(np.concatenate([instance + noise, label]))

In [3]:
train_data = data[:int(0.7*n)]
val_data = data[int(0.7*n):int(0.8*n)]
test_data = data[int(0.8*n):]

train_tensor = torch.tensor(train_data)
val_tensor = torch.tensor(val_data)
test_tensor = torch.tensor(test_data)

/tmp/ipykernel_7172/341918777.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  train_tensor = torch.tensor(train_data)


In [4]:
train_tensor = torch.tensor(train_data)
val_tensor = torch.tensor(val_data)
test_tensor = torch.tensor(test_data)

In [5]:
class Vertexes(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        X = self.data[idx, :3]
        y = self.data[idx, 3]
        return X, y

In [6]:
BATCH_SIZE = 128

train_dataloader = DataLoader(Vertexes(train_tensor), batch_size=BATCH_SIZE)
val_dataloader = DataLoader(Vertexes(val_tensor), batch_size=BATCH_SIZE)
test_dataloader = DataLoader(Vertexes(test_tensor), batch_size=BATCH_SIZE)

device = ( "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu")

class Rosemblatt(nn.Module):
    def __init__(self):
        super(Rosemblatt, self).__init__()
        self.perceptron = nn.Linear(3, 8, dtype=torch.float64)

    def forward(self, x):
        x = self.perceptron(x)
        output = F.softmax(x, dim=1, dtype=torch.float64)
        return output

model = Rosemblatt().to(device)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
     
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y.type(torch.LongTensor)
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() 
    test_loss, correct = 0, 0
    with torch.no_grad(): 
        for X, y in dataloader:
            y = y.type(torch.LongTensor)
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
EPOCHS = 300
for t in range(EPOCHS):
    print(f"Epoch {t+1} -----------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
print("-----------------Done-----------------")

Epoch 1 -----------------
Accuracy: 12.4%, Avg loss: 2.092606 

Epoch 2 -----------------
Accuracy: 12.4%, Avg loss: 2.087980 

Epoch 3 -----------------
Accuracy: 12.4%, Avg loss: 2.083317 

Epoch 4 -----------------
Accuracy: 13.0%, Avg loss: 2.078627 

Epoch 5 -----------------
Accuracy: 13.0%, Avg loss: 2.073924 

Epoch 6 -----------------
Accuracy: 13.0%, Avg loss: 2.069221 

Epoch 7 -----------------
Accuracy: 13.1%, Avg loss: 2.064528 

Epoch 8 -----------------
Accuracy: 13.3%, Avg loss: 2.059848 

Epoch 9 -----------------
Accuracy: 14.1%, Avg loss: 2.055180 

Epoch 10 -----------------
Accuracy: 14.7%, Avg loss: 2.050516 

Epoch 11 -----------------
Accuracy: 15.7%, Avg loss: 2.045844 

Epoch 12 -----------------
Accuracy: 16.6%, Avg loss: 2.041147 

Epoch 13 -----------------
Accuracy: 18.1%, Avg loss: 2.036407 

Epoch 14 -----------------
Accuracy: 19.2%, Avg loss: 2.031607 

Epoch 15 -----------------
Accuracy: 20.4%, Avg loss: 2.026729 

Epoch 16 -----------------
Accurac

In [10]:
test(test_dataloader, model, loss_fn)

Accuracy: 100.0%, Avg loss: 1.331580 

